In [ ]:
!pip install pmdarima

In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

from pmdarima.arima import auto_arima, ADFTest
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
annual = pd.read_csv('../input/gold-prices/annual_csv.csv')
month = pd.read_csv('../input/gold-prices/monthly_csv.csv')

In [ ]:
month.head()

In [ ]:
fig = px.line(month, x='Date', y="Price")
fig.show()

In [ ]:
plot_acf(month['Price'])
plot_pacf(month['Price'])

In [ ]:
fig = px.line(annual, x='Date', y="Price")
fig.show()

## ADF Test

#### ADF Test is used for check stationary
##### If p-value is less than 0.05, this timeseries data is stationary
##### If ADF Statistics is less than Critical value, this timeseries data is stationary

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
def print_adfuller(inputseries):
    result = adfuller(inputseries)
    print("ADF Statistic : %f" % result[0])
    print("p-value : %f" % result[1])
    print('Critical values : ')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key ,value))

In [ ]:
print_adfuller(month['Price'])

In [ ]:
adf_test = ADFTest(alpha=0.05)
adf_test.should_diff(month['Price'])

In [ ]:
diff_1 = month['Price'].diff(periods=1)
diff_1 = diff_1.iloc[1:]

In [ ]:
print_adfuller(diff_1)

In [ ]:
fig = px.line(diff_1)
fig.show()

In [ ]:
auto_arima_model = auto_arima(month['Price'], p=0, d=1, start_q=0, max_p=3, max_d=3, max_q=3,start_P=0, D=1,start_Q=0,max_P=3,
                              max_D=3, max_Q=3, m=4, seasonal=True, error_action='warn', trace=True, stepwise=True,
                              random_state=20)

In [ ]:
month.tail()

In [ ]:
pred_idx = list(range(847,857))
pred_idx

In [ ]:
pred = pd.DataFrame(auto_arima_model.predict(n_periods=10), columns=['Pred'], index=pred_idx)

In [ ]:
pred

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.plot(month['Price'],label='Training')
plt.plot(pred['Pred'],label='Prediction')
plt.legend(loc='best')
plt.show()